In [28]:
import pandas as pd
import os
from dotenv import load_dotenv


In [34]:
# Load your DataFrame
df = pd.read_csv("data/data_with_best_model_sentiment_analysis.csv")  # Uncomment and modify if needed
df1 = df[["tokenized_reviews"]].sample(n=200, random_state=42).reset_index(drop=True)  # Sample 200 reviews

In [35]:
df1 = df1.dropna()


In [36]:
df1.shape

(199, 1)

In [39]:
import os
import json
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai import ChatMistralAI
from collections import Counter
from time import sleep



# Set API key
load_dotenv()
mistral_api_key = os.getenv("MISTRAL_API_KEY")


# Prompt setup
sys_prompt = """
Follow instructions below and extract intents from a customer review as a json string. DO NOT include any notes or additional information in the output.

### Instructions:
- **Identify each distinct intent** in the review as "intent".** 
- **intent musk qualify the customer experience the most objectively possible** 
- **Be simple and synthetic. Use only aphabetique characteres. Only in lowcase. Use correct english.**
- **Return a list of intents a JSON string.**

### Output Format
{{format_instructions}}

### Example Review:
"It'd McDonalds. It is what it is as far as the food and atmosphere go. The staff here does make a difference. They are all friendly, accommodating and always smiling. Makes for a more pleasant experience than many other fast food places."
### Example Output (JSON format):
{{example_output}}

### Review to analyze:
{{review}}
"""

format_instructions = {
  "intents": ['']
}

example_output = {
  "intents": ["atmosphere","staff","kindness"]
}

# Prompt template and model
template = ChatPromptTemplate.from_messages([
    ("system", sys_prompt),
    ("user", "{review}"),
])

model = ChatMistralAI(model="mistral-small-latest")
chain = template | model

# Output file for saving intermediate results
output_path = "intent_results.jsonl"
processed_indices = set()

# Load already processed if exists
if os.path.exists(output_path):
    with open(output_path, "r") as f:
        for line in f:
            data = json.loads(line)
            processed_indices.add(data["index"])

# Main loop with saving after each call
for i, row in df1.iterrows():
    if i in processed_indices:
        continue  # Skip already processed

    review_text = row["tokenized_reviews"]
    try:
        response = chain.invoke({"review": review_text})
        intents = json.loads(response.content)

        # Save the result
        with open(output_path, "a") as f:
            f.write(json.dumps({"index": i, "intents": intents}) + "\n")

        print(f"Processed review {i}, intents : {intents}")

        sleep(10)  # Add delay if needed to respect rate limits

    except Exception as e:
        print(f"Error processing review {i}: {e}")
        continue # to skip and log errors instead





Error processing review 0: Expecting value: line 1 column 1 (char 0)
Processed review 1, intents : [{'intent': 'eat'}, {'intent': 'rest'}]
Error processing review 4: Expecting value: line 1 column 1 (char 0)
Error processing review 5: Error response 429 while fetching https://api.mistral.ai/v1/chat/completions: {"message":"Requests rate limit exceeded"}
Error processing review 6: Error response 429 while fetching https://api.mistral.ai/v1/chat/completions: {"message":"Requests rate limit exceeded"}
Error processing review 8: Error response 429 while fetching https://api.mistral.ai/v1/chat/completions: {"message":"Requests rate limit exceeded"}
Error processing review 9: Error response 429 while fetching https://api.mistral.ai/v1/chat/completions: {"message":"Requests rate limit exceeded"}
Error processing review 10: Error response 429 while fetching https://api.mistral.ai/v1/chat/completions: {"message":"Requests rate limit exceeded"}
Error processing review 11: Error response 429 whil

In [40]:
# --- Count intents ---
from collections import Counter
import json

intent_counter = Counter()

with open("intent_results.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)
        intents_data = data["intents"]

        # Normalize format
        if isinstance(intents_data, dict) and "intents" in intents_data:
            intents_list = intents_data["intents"]
        elif isinstance(intents_data, list):
            intents_list = intents_data
        else:
            continue  # Skip unexpected formats

        # Count intents
        for item in intents_list:
            if isinstance(item, dict):
                intent = item.get("intent", "").strip()
            elif isinstance(item, str):
                intent = item.strip()
            else:
                continue  # skip if it's neither dict nor string

            if intent:
                intent_counter[intent] += 1

# Print results
print("\nIntent Frequencies:")
for intent, count in intent_counter.most_common():
    print(f"{intent}: {count}")


Intent Frequencies:
eat: 3
rude staff: 3
fast food: 2
bad food: 2
bad service: 2
rest: 2
cleanliness: 2
order fast: 1
food ready fast: 1
cheap food: 1
eats: 1
want food: 1
want happy meal: 1
dont want toy: 1
avoid: 1
unhygienic staff: 1
long drive: 1
prefer other restaurants: 1
request information: 1
report problem: 1
complain: 1
long wait: 1
unprofessional crew: 1
earn points: 1
app usage: 1
always busy: 1
moves fast: 1
good food: 1
order delivery: 1
starving: 1
energy: 1
happy: 1
feel good: 1
find everything: 1
missing: 1
dirty restroom: 1
slow service: 1
enjoy food: 1
appreciate service: 1
order quality: 1
food temperature: 1
long wait time: 1
poor communication: 1
poor atmosphere: 1
good quality: 1
cheap price: 1
order food: 1
experience atmosphere: 1
staff friendliness: 1
staff accommodation: 1
staff smiling: 1
pleasant experience: 1
compare fast food places: 1
price: 1
convenience: 1
location: 1
request_help: 1
order_taken: 1
staff_walked_away: 1
bad experience: 1
staff behavior

In [39]:
# --- Count intents ---
from collections import Counter
import json

intent_counter = Counter()

with open("intent_results.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)
        intents_data = data["intents"]

        # Normalize format
        if isinstance(intents_data, dict) and "intents" in intents_data:
            intents_list = intents_data["intents"]
        elif isinstance(intents_data, list):
            intents_list = intents_data
        else:
            continue  # Skip unexpected formats

        # Count intents
        for item in intents_list:
            if isinstance(item, dict):
                intent = item.get("intent", "").strip()
            elif isinstance(item, str):
                intent = item.strip()
            else:
                continue  # skip if it's neither dict nor string

            if intent:
                intent_counter[intent] += 1

# Print results
print("\nIntent Frequencies:")
for intent, count in intent_counter.most_common():
    print(f"{intent}: {count}")


Intent Frequencies:
Customer Service: 4
Staff Behavior: 4
Food Quality: 3
Operating Hours: 3
Service: 3
complain: 3
price: 3
Order Accuracy: 2
Location: 2
Dining Experience: 2
Cleanliness: 2
Order_Accuracy: 2
Refund_Policy: 2
Food_Quality: 2
Recommendation: 2
service: 2
dirty: 2
order: 2
bad service: 2
friendly staff: 2
Drive-thru experience: 1
General Experience: 1
Safety Concerns: 1
Advice: 1
Receipt Issue: 1
Order Resolution: 1
Location Size: 1
Staffing: 1
Menu Options: 1
general_experience: 1
Security: 1
Decor: 1
drive_through_experience: 1
service_experience: 1
Customer Loyalty: 1
Product Consistency: 1
Language Barrier: 1
Staff Attitude: 1
Long Wait Time: 1
Left Without Ordering: 1
Order: 1
Manager_Interaction: 1
Wait_Time: 1
Service Quality: 1
Product Availability: 1
convenience: 1
satisfaction: 1
Affordability: 1
Overall Experience: 1
Staff Friendliness: 1
Busyness: 1
Order Issue: 1
Service Speed: 1
Closing Time: 1
Dissatisfaction with menu: 1
app_rewards: 1
wait_time: 1
overa

In [41]:
import os
import json
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai import ChatMistralAI
from collections import Counter
from time import sleep



# Set API key
os.environ["MISTRAL_API_KEY"] = "ewfejjYMM0oUdCk4ndJcFTDa5VwVjVHi"

# Prompt setup
sys_prompt = """
Follow instructions below and extract intents from a customer review as a json string. DO NOT include any notes or additional information in the output.

### Instructions:
- **Identify each distinct intent** in the review as "intent". Be simple and synthetic. Use only aphabetique characteres. Only in lowcase.
- **Summarize the text** associated with each intent as "text_summary".
- **Classify the sentiment** (Positive, Negative, or Neutral) of each intent as "sentiment".
- **Return a list of intents a JSON string.**

### Output Format
{{format_instructions}}

### Example Review:
"It'd McDonalds. It is what it is as far as the food and atmosphere go. The staff here does make a difference. They are all friendly, accommodating and always smiling. Makes for a more pleasant experience than many other fast food places."
### Example Output (JSON format):
{{example_output}}

### Review to analyze:
{{review}}
"""

format_instructions = {
  "intents": [
    {"intent": "", "text_summary": "", "sentiment": ""},
  ]
}

example_output = {
  "intents": [
    {
      "intent": "Atmosphere",
      "text_summary": "It'd McDonalds. It is what it is as far as the food and atmosphere go.",
      "sentiment": "Neutral",
    },
    {
      "intent": "Staff",
      "text_summary": "The staff here does make a difference. They are all friendly, accommodating and always smiling.",
      "sentiment": "Positive",
    }
  ]
}

# Prompt template and model
template = ChatPromptTemplate.from_messages([
    ("system", sys_prompt),
    ("user", "{review}"),
])

model = ChatMistralAI(model="mistral-small-latest")
chain = template | model

# Output file for saving intermediate results
output_path = "intent_results.jsonl"
processed_indices = set()

# Load already processed if exists
if os.path.exists(output_path):
    with open(output_path, "r") as f:
        for line in f:
            data = json.loads(line)
            processed_indices.add(data["index"])

# Main loop with saving after each call
for i, row in df.iterrows():
    if i in processed_indices:
        continue  # Skip already processed

    review_text = row["review"]
    try:
        response = chain.invoke({"review": review_text})
        intents = json.loads(response.content)

        # Save the result
        with open(output_path, "a") as f:
            f.write(json.dumps({"index": i, "intents": intents}) + "\n")

        print(f"Processed review {i}, intents : {intents}")

        sleep(1)  # Add delay if needed to respect rate limits

    except Exception as e:
        print(f"Error processing review {i}: {e}")
        continue # to skip and log errors instead





Error processing review 7: Expecting value: line 1 column 1 (char 0)


Error processing review 8: Expecting value: line 1 column 1 (char 0)
Error processing review 10: Expecting value: line 1 column 1 (char 0)
Error processing review 11: Expecting value: line 1 column 1 (char 0)
Error processing review 12: Expecting value: line 1 column 1 (char 0)
Error processing review 15: Error response 429 while fetching https://api.mistral.ai/v1/chat/completions: {"message":"Requests rate limit exceeded"}
Error processing review 16: Error response 429 while fetching https://api.mistral.ai/v1/chat/completions: {"message":"Requests rate limit exceeded"}
Processed review 17, intents : [{'intent': 'app_rewards', 'text_summary': 'Earning points through the mobile app', 'sentiment': 'Positive'}]
Error processing review 18: Expecting value: line 1 column 1 (char 0)
Error processing review 19: Error response 429 while fetching https://api.mistral.ai/v1/chat/completions: {"message":"Requests rate limit exceeded"}
Error processing review 24: Error response 429 while fetching h

In [42]:
# --- Count intents ---
from collections import Counter
import json

intent_counter = Counter()

with open("intent_results.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)
        intents_data = data["intents"]

        # Normalize format
        if isinstance(intents_data, dict) and "intents" in intents_data:
            intents_list = intents_data["intents"]
        elif isinstance(intents_data, list):
            intents_list = intents_data
        else:
            continue  # Skip unexpected formats

        # Count intents
        for item in intents_list:
            intent = item.get("intent", "").strip()
            if intent:
                intent_counter[intent] += 1

# Print results
print("\nIntent Frequencies:")
for intent, count in intent_counter.most_common():
    print(f"{intent}: {count}")


Intent Frequencies:
Customer Service: 4
Staff Behavior: 4
Food Quality: 3
Operating Hours: 3
Service: 3
Order Accuracy: 2
Location: 2
Dining Experience: 2
Cleanliness: 2
Order_Accuracy: 2
Refund_Policy: 2
Food_Quality: 2
Recommendation: 2
service: 2
Drive-thru experience: 1
General Experience: 1
Safety Concerns: 1
Advice: 1
Receipt Issue: 1
Order Resolution: 1
Location Size: 1
Staffing: 1
Menu Options: 1
general_experience: 1
Security: 1
Decor: 1
drive_through_experience: 1
service_experience: 1
Customer Loyalty: 1
Product Consistency: 1
Language Barrier: 1
Staff Attitude: 1
Long Wait Time: 1
Left Without Ordering: 1
Order: 1
Manager_Interaction: 1
Wait_Time: 1
Service Quality: 1
Product Availability: 1
convenience: 1
satisfaction: 1
Affordability: 1
Overall Experience: 1
Staff Friendliness: 1
Busyness: 1
Order Issue: 1
Service Speed: 1
Closing Time: 1
Dissatisfaction with menu: 1
app_rewards: 1
wait_time: 1
overall_experience: 1
food_quality: 1
Service Refusal: 1
experience: 1
Behavi

In [8]:
!pip install langchain_mistralai

In [29]:
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_mistralai import ChatMistralAI
from langchain_core.messages import HumanMessage
from pydantic import ValidationError  # Import ValidationError

# Set the environment variable for the API key
api_key = "ewfejjYMM0oUdCk4ndJcFTDa5VwVjVHi"
os.environ["MISTRAL_API_KEY"] = api_key

# Debug: Print the API key to ensure it is set correctly
print(f"API Key: {os.environ.get('MISTRAL_API_KEY')}")

# Prompt template
sys_prompt = """
Follow instructions below and extract intents from a customer review as a json string. DO NOT include any notes or additional information in the output.

### Instructions:
- **Identify each distinct intent** in the review as "intent". The review may contain multiple distinct intents related to different aspects of the customer's experience (e.g., Hygiene, Food quality, Staff, Incomplete order...). Note that the list in e.g. is not exhaustive. Please make sure to identify the most appropritate intent.
- **Summarize the text** associated with each intent as "text_summary".
- **Classify the sentiment** (Positive, Negative, or Neutral) of each intent as "sentiment".
- **Return a list of intents a JSON string.** Follow the output format and use example output below as a reference. Make sure the JSON string is COMPLETE. Do not include additional information.

### Output Format
{{format_instructions}}

### Example Review:
"It'd McDonalds. It is what it is as far as the food and atmosphere go. The staff here does make a difference. They are all friendly, accommodating and always smiling. Makes for a more pleasant experience than many other fast food places."
### Example Output (JSON format):
{{example_output}}

### Review to analyze:
{{review}}
"""

# Format des instructions
format_instructions = {
  "intents": [
    {
      "intent": "",
      "text_summary": "",
      "sentiment": "",
    },
    {
      "intent": "",
      "text_summary": "",
      "sentiment": "",
    }
  ]
}

example_output = {
  "intents": [
    {
      "intent": "Atmosphere",
      "text_summary": "It'd McDonalds. It is what it is as far as the food and atmosphere go.",
      "sentiment": "Neutral",
    },
    {
      "intent": "Staff",
      "text_summary": "The staff here does make a difference. They are all friendly, accommodating and always smiling.",
      "sentiment": "Positive",
    }
  ]
}

# Création du modèle de prompt
template = ChatPromptTemplate.from_messages([
    ("system", sys_prompt),
    ("user", "{review}"),
])

# Let's instantiate a model
try:
    model = ChatMistralAI(model="mistral-small-latest")
except ValidationError as e:
    print(f"ValidationError: {e}")
    # Try an alternative model name if available
    try:
        model = ChatMistralAI(model="mistral-small")
    except ValidationError as e:
        print(f"ValidationError: {e}")
        raise e  # Re-raise the exception if both model names fail

# This is the simple chain
answer = template | model

# Get the response
response = answer.invoke({"review": 'If I could give this location a zero on customer service, I would. Pulled into drive through at 8:07 and placed a simple order. I order a regular hamburg and cookies. Once I paid and pulled up to the window to pick up, I was told to pull up to the #1 space. I sat there and sat there and.........25 minutes later, no order. I went in and no one was at the counter. I yelled "hello" three times. A young lady came up asking who was handling the front counter. A young girl appeared. To make this complaint shorter, I showed my receipt and another 5 minutes later I got my order. There was another order with a drink sitting on the counter that never got to the person who ordered it. After getting home, I had no regular hamburg but had cookies. Everyone, don\'t even bother going to this location again. It\'s a horrible experience every time as you can see by other reviews.'})
print(response.content)


API Key: ewfejjYMM0oUdCk4ndJcFTDa5VwVjVHi


[
  {
    "intent": "Incomplete order",
    "text_summary": "I ordered a regular hamburger and cookies, but I only received the cookies.",
    "sentiment": "Negative"
  },
  {
    "intent": "Staff",
    "text_summary": "No one was at the counter when I went in. A young lady and a young girl eventually appeared, but the service was poor.",
    "sentiment": "Negative"
  },
  {
    "intent": "Wait time",
    "text_summary": "I waited 25 minutes for my order in the drive-thru, and an additional 5 minutes inside the restaurant.",
    "sentiment": "Negative"
  },
  {
    "intent": "Customer service",
    "text_summary": "The customer service was poor. I had to yell 'hello' three times to get attention, and the order was mishandled.",
    "sentiment": "Negative"
  },
  {
    "intent": "General experience",
    "text_summary": "The overall experience was horrible, and I advise others not to go to this location.",
    "sentiment": "Negative"
  }
]


In [24]:
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_mistralai import ChatMistralAI
from langchain_core.messages import HumanMessage
from pydantic import ValidationError  # Import ValidationError

# Set the environment variable for the API key
api_key = "ma_clée"
os.environ["MISTRAL_API_KEY"] = api_key

# Debug: Print the API key to ensure it is set correctly
print(f"API Key: {os.environ.get('MISTRAL_API_KEY')}")

# Prompt template
sys_prompt = """
Follow instructions below and extract intents from a customer review as a json string. DO NOT include any notes or additional information in the output.

### Instructions:
- **Identify each distinct intent** in the review as "intent". The review may contain multiple distinct intents related to different aspects of the customer's experience (e.g., Hygiene, Food quality, staff...).
- **Summarize the text** associated with each intent as "text_summary".
- **Classify the sentiment** (Positive, Negative, or Neutral) of each intent as "sentiment".
- **Return a list of intents a JSON string.** Follow the output format and use example output below as a reference. Make sure the JSON string is COMPLETE. Do not include additional information.

### Output Format
{{format_instructions}}

### Example Review:
"It'd McDonalds. It is what it is as far as the food and atmosphere go. The staff here does make a difference. They are all friendly, accommodating and always smiling. Makes for a more pleasant experience than many other fast food places."
### Example Output (JSON format):
{{example_output}}

### Review to analyze:
{{review}}
"""

# Format des instructions
format_instructions = {
  "intents": [
    {
      "intent": "",
      "text_summary": "",
      "sentiment": "",
    },
    {
      "intent": "",
      "text_summary": "",
      "sentiment": "",
    }
  ]
}

example_output = {
  "intents": [
    {
      "intent": "Atmosphere",
      "text_summary": "It'd McDonalds. It is what it is as far as the food and atmosphere go.",
      "sentiment": "Neutral",
    },
    {
      "intent": "Staff",
      "text_summary": "The staff here does make a difference. They are all friendly, accommodating and always smiling.",
      "sentiment": "Positive",
    }
  ]
}

template = ChatPromptTemplate.from_messages([
    ("system", sys_prompt),
    ("user", "{review}"),
])


# Don't forget to change the key
%env MISTRAL_API_KEY="ewfejjYMM0oUdCk4ndJcFTDa5VwVjVHi"
# Let's instanciate a model
model = ChatMistralAI(model="mistral-small-latest")


# This is the simple chain
answer = template | model

# Get the response
response = answer.invoke({"review": "The service was horrible, I had the worst experience ever, the place was dirty and they forgot my fries !!!!"})
print(response.content)



API Key: ma_clée
env: MISTRAL_API_KEY="ewfejjYMM0oUdCk4ndJcFTDa5VwVjVHi"


HTTPStatusError: Error response 401 while fetching https://api.mistral.ai/v1/chat/completions: {
  "message":"Unauthorized",
  "request_id":"f1d4794b80f8ad1b12e8471eb2f26743"
}

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_mistralai import ChatMistralAI

prompt_template = """
Follow instructions below and extract intents from a customer review as a json string. DO NOT include any notes or additional information in the output.

### Instructions:
- **Identify each distinct intent** in the review as "intent". The review may contain multiple distinct intents related to different aspects of the customer's experience (e.g., Hygiene, Food quality, staff...).
- **Summarize the text** associated with each intent as "text_summary".
- **Classify the sentiment** (Positive, Negative, or Neutral) of each intent as "sentiment".
- **Return a list of intents a JSON string.** Follow the output format and use example output below as a reference. Make sure the JSON string is COMPLETE. Do not include additional information.

### Output Format
{{format_instructions}}

### Example Review:
"It'd McDonalds. It is what it is as far as the food and atmosphere go. The staff here does make a difference. They are all friendly, accommodating and always smiling. Makes for a more pleasant experience than many other fast food places."
### Example Output (JSON format):
{{example_output}}

### Review to analyze:
{{review}}
"""

# Format des instructions
format_instructions = {
  "intents": [
    {
      "intent": "",
      "text_summary": "",
      "sentiment": "",
    },
    {
      "intent": "",
      "text_summary": "",
      "sentiment": "",
    }
  ]
}

example_output = {
  "intents": [
    {
      "intent": "Atmosphere",
      "text_summary": "It'd McDonalds. It is what it is as far as the food and atmosphere go.",
      "sentiment": "Neutral",
    },
    {
      "intent": "Staff",
      "text_summary": "The staff here does make a difference. They are all friendly, accommodating and always smiling.",
      "sentiment": "Positive",
    }
  ]
}

# Création du modèle de prompt
prompt = ChatPromptTemplate.from_messages([
    ("input_variables",review),
    ('template',prompt_template),
    ('partial_variables',{
      "format_instructions": format_instructions,
      "example_output": example_output
    })
  ]
)



# Don't forget to change the key
%env MISTRAL_API_KEY=ma_clée
# Let's instanciate a model
model = ChatMistralAI(model="mistral-small-latest")


# This is the simple chain
answer = template | model

# Get the response
response = answer.invoke({"review": "The service was horrible, I had the worst experience ever, the place was dirty and they forgot my fries !!!!"})
print(response.content)

ValueError: Unexpected message type: input_variables. Use one of 'human', 'user', 'ai', 'assistant', or 'system'.

In [7]:
from langchain.prompts import PromptTemplate
import mistral  
import mistralai  

mistralai.api_key = ma_clée
# Définition des outils (si nécessaire pour votre application)
tools = [
  {
    "type": "function",
    "function": {
        "name": "extracts_intents",
        "parameters": {
          "type": "object",
          "properties": {
            "intents": {
              "type": "array",
              "description": "List of intents identified from the customer review",
              "items": {
                "type": "object",
                "properties": {
                  "intent": {
                    "type": "string",
                    "description": "Description of the identified intent"
                  },
                  "text_summary": {
                    "type": "string",
                    "description": "Summary of the intent"
                  },
                  "sentiment": {
                    "type": "string",
                    "enum": ["Positive", "Negative", "Neutral"],
                    "description": "Sentiment of the intent"
                  },
                },
                "required": ["intent", "text_summary", "sentiment"]
              }
            }
          }
        }
    }
  }
]

# Exemple de sortie
example_output = {
  "intents": [
    {
      "intent": "Drive-thru experience",
      "text_summary": "I went to the drive-thru to pass my order and I was satisfied.",
      "sentiment": "Positive"
    },
    {
      "intent": "Hygiene",
      "text_summary": "The place is dirty.",
      "sentiment": "Negative"
    },
    {
      "intent": "Food quality",
      "text_summary": "The food quality is below average.",
      "sentiment": "Negative"
    },
    {
      "intent": "Staff",
      "text_summary": "The staff is polite, smiling and efficient.",
      "sentiment": "Positive"
    },
    {
      "intent": "Order",
      "text_summary": "They is always something missing",
      "sentiment": "Negative"
    },
    {
      "intent": "Temperature of the food",
      "text_summary": "My fries were cold",
      "sentiment": "Negative"
    },
    {
      "intent": "Location",
      "text_summary": "It's McDonald's. Nothing special",
      "sentiment": "Neutral"
    },
  ]
}

# Modèle de prompt
prompt_template = """
Follow instructions below and extract intents from a customer review as a json string. DO NOT include any notes or additional information in the output.

### Instructions:
- **Identify each distinct intent** in the review as "intent". The review may contain multiple distinct intents related to different aspects of the customer's experience (e.g., Hygiene, Food quality, staff...).
- **Summarize the text** associated with each intent as "text_summary".
- **Classify the sentiment** (Positive, Negative, or Neutral) of each intent as "sentiment".
- **Return a list of intents a JSON string.** Follow the output format and use example output below as a reference. Make sure the JSON string is COMPLETE. Do not include additional information.

### Output Format
{{format_instructions}}

### Example Review:
"It'd McDonalds. It is what it is as far as the food and atmosphere go. The staff here does make a difference. They are all friendly, accommodating and always smiling. Makes for a more pleasant experience than many other fast food places."
### Example Output (JSON format):
{{example_output}}

### Review to analyze:
{{review}}
"""

# Format des instructions
format_instructions = {
  "intents": [
    {
      "intent": "",
      "text_summary": "",
      "sentiment": "",
    },
    {
      "intent": "",
      "text_summary": "",
      "sentiment": "",
    }
  ]
}

example_output = {
  "intents": [
    {
      "intent": "Atmosphere",
      "text_summary": "It'd McDonalds. It is what it is as far as the food and atmosphere go.",
      "sentiment": "Neutral",
    },
    {
      "intent": "Staff",
      "text_summary": "The staff here does make a difference. They are all friendly, accommodating and always smiling.",
      "sentiment": "Positive",
    }
  ]
}

# Création du modèle de prompt
prompt = PromptTemplate(
    input_variables=["review"],
    template=prompt_template,
    partial_variables={
        "format_instructions": format_instructions,
        "example_output": example_output
    }
)

def get_request(prompt, review):
    formatted_prompt = prompt.format(review=review)
    client = mistralai(api_key=mistralai.api_key)  # Utilisez le client Mistral
    response = client.generate(
        model="mistral-model",  # Remplacez par le nom du modèle Mistral
        prompt=formatted_prompt,
        max_tokens=150,
        temperature=0.7,
    )
    return response['choices'][0]['text'].strip()


# Exemple d'utilisation
review = 'I repeat my order 3 times in the drive thru, and she still manage to mess it up , it was suppose to be a large meal double filet of fish with large fries , no cheese . It was all wrong , they either need to pay close attention to the order being made , understand English or they need not to work at a drive thru'
output = get_request(prompt, review)
print(output)


TypeError: 'module' object is not callable